In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# CSVデータを読み込む
try:
    df = pd.read_csv("../../result.csv")
except FileNotFoundError:
    print("エラー: ../../result.csv が見つかりませんでした。ファイルパスを確認してください。")
    exit()

print("CSVデータが正常に読み込まれました。\n")

# 保存用ディレクトリ作成
output_dir = "comparison_plots"
os.makedirs(output_dir, exist_ok=True)

def save_plot(plt, dataset_name, title_suffix):
    """グラフを保存し表示するヘルパー関数"""
    safe_dataset_name = "".join(c if c.isalnum() else "_" for c in str(dataset_name))
    safe_title_suffix = "".join(c if c.isalnum() else "_" for c in str(title_suffix)).replace("__", "_")
    filename = f"{safe_dataset_name}_{safe_title_suffix}.png"
    filepath = os.path.join(output_dir, filename)
    try:
        plt.savefig(filepath)
        print(f"グラフを {filepath} として保存しました。")
    except Exception as e:
        print(f"グラフの保存中にエラーが発生しました ({filepath}): {e}")
    plt.show()

def plot_dataset_overall(df, dataset_name):
    """データセット全体のモデル比較グラフを作成する関数"""
    plt.figure(figsize=(12, 7))
    
    models_in_dataset = df[df['dataset'] == dataset_name]['model'].unique()
    
    plotted_something = False
    for model in models_in_dataset:
        for bins in [1, 10]:
            subset = df[(df['model'] == model) &
                        (df['dataset'] == dataset_name) &
                        (df['bins'] == bins)]
            if not subset.empty:
                subset = subset.sort_values('accum_t_ms')
                label = f"{model} (bins={bins})"
                plt.plot(subset['accum_t_ms'], subset['mAP'], marker='o', linestyle='-', label=label)
                plotted_something = True

    if plotted_something:
        plt.title(f"Dataset: {dataset_name} - Overall Model Comparison", fontsize=16)
        plt.xlabel("Accumulation Time (ms)", fontsize=14)
        plt.ylabel("mAP", fontsize=14)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        save_plot(plt, dataset_name, "overall_comparison")
    else:
        print(f"データセット {dataset_name} にはプロットするデータがありませんでした（Overall）。")
        plt.close()

def plot_specific_model_comparison(df, dataset_name, models_to_compare, comparison_title):
    """指定されたモデルペアの比較グラフを作成する関数"""
    plt.figure(figsize=(10, 6))
    
    plotted_something = False
    for model in models_to_compare:
        # 指定されたモデルがデータセットに存在するか確認
        if not df[(df['model'] == model) & (df['dataset'] == dataset_name)].empty:
            for bins in [1, 10]:
                subset = df[(df['model'] == model) &
                            (df['dataset'] == dataset_name) &
                            (df['bins'] == bins)]
                if not subset.empty:
                    subset = subset.sort_values('accum_t_ms')
                    label = f"{model} (bins={bins})"
                    plt.plot(subset['accum_t_ms'], subset['mAP'], marker='o', linestyle='-', label=label)
                    plotted_something = True
        else:
            print(f"警告: モデル '{model}' はデータセット '{dataset_name}' に存在しないため、'{comparison_title}' の比較から除外されます。")

    if plotted_something:
        plt.title(f"Dataset: {dataset_name} - {comparison_title}", fontsize=16)
        plt.xlabel("Accumulation Time (ms)", fontsize=14)
        plt.ylabel("mAP", fontsize=14)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        save_plot(plt, dataset_name, comparison_title)
    else:
        print(f"データセット {dataset_name} でモデル {models_to_compare} のデータが見つからなかったため、グラフ「{comparison_title}」は生成されませんでした。")
        plt.close()

# --- グラフ生成処理 ---
datasets_in_df = df['dataset'].unique()

# Gen1 データセットの処理
if 'gen1' in datasets_in_df:
    print("\n--- Processing gen1 dataset ---")
    plot_dataset_overall(df, 'gen1')
else:
    print("\n警告: gen1 データセットはCSVファイル内に見つかりませんでした。")

# Gen4 データセットの処理
if 'gen4' in datasets_in_df:
    print("\n--- Processing gen4 dataset ---")
    # plot_dataset_overall(df, 'gen4') # Gen4の全体比較も必要ならコメントアウトを外す
    plot_specific_model_comparison(df, 'gen4', ['YOLOX', 'YOLOX-LSTM'], "YOLOX vs YOLOX-LSTM (LSTM)")
    plot_specific_model_comparison(df, 'gen4', ['YOLOX-LSTM', 'RVT'], "YOLOX-LSTM vs RVT (Attention vs CNN)")
    # RVT-Track が gen4 にのみ存在すると仮定して比較
    if 'RVT-Track' in df[df['dataset'] == 'gen4']['model'].unique() and 'RVT' in df[df['dataset'] == 'gen4']['model'].unique():
        plot_specific_model_comparison(df, 'gen4', ['RVT', 'RVT-Track'], "RVT vs RVT-Track (Tracking)")
    elif not ('RVT-Track' in df[df['dataset'] == 'gen4']['model'].unique()):
         print("情報: モデル 'RVT-Track' はデータセット 'gen4' に存在しないため、RVTとの比較はスキップされます。")
    elif not ('RVT' in df[df['dataset'] == 'gen4']['model'].unique()):
         print("情報: モデル 'RVT' はデータセット 'gen4' に存在しないため、RVT-Trackとの比較はスキップされます。")
else:
    print("\n警告: gen4 データセットはCSVファイル内に見つかりませんでした。")

# DSEC データセットの処理
if 'dsec' in datasets_in_df:
    print("\n--- Processing dsec dataset ---")
    # plot_dataset_overall(df, 'dsec') # DSECの全体比較も必要ならコメントアウトを外す
    plot_specific_model_comparison(df, 'dsec', ['YOLOX', 'YOLOX-LSTM'], "YOLOX vs YOLOX-LSTM (LSTM)")
    plot_specific_model_comparison(df, 'dsec', ['YOLOX-LSTM', 'RVT'], "YOLOX-LSTM vs RVT (Attention vs CNN)")
else:
    print("\n警告: dsec データセットはCSVファイル内に見つかりませんでした。")

print("\n全てのグラフ生成処理が要求に従い完了しました。")